In [3]:
#import dependencies
import pandas as pd

In [4]:
#set path for csv file
csvpath = "Resources/purchase_data.csv"
#import the purchase_data.csv file as a DataFrame
purchase_data_df = pd.read_csv(csvpath)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#calculate the Player Count (PC)
total_players = len(purchase_data_df["SN"].unique())
total_players

576

In [43]:
#place Player Count into a summary DataFrame
summary_table_PC = pd.DataFrame({"Total Players": [total_players]})
player_demo = purchase_data_df.loc[:, ["Gender", "SN", "Age"]]
player_demo = player_demo.drop_duplicates()
summary_table_PC

,Total Players
0,576


In [7]:
#Purchasing Analysis (PA)
#need to find: number of unique items, average price, total # of purchases, total revenue
#be careful of ordering

#calculate the number of unique items
unique_items = len(purchase_data_df["Item ID"].unique())
print(unique_items)
#calculate the total revenue
total_revenue = sum(purchase_data_df["Price"])

#calculate the total number of purchases
total_number_of_purchases = len(purchase_data_df["Price"])

#calculate the average price 
average_price = total_revenue / total_number_of_purchases

183


In [8]:
# Place all of the Purchasing Analysis data found into a summary DataFrame
summary_table_PA = pd.DataFrame({"Number of Unique Items": [unique_items],
                                                  "Average Price": [average_price],
                                                  "Number of Purchases": [total_number_of_purchases],
                                                  "Total Revenue": [total_revenue]                                                  
                                                 })
summary_table_PA.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [9]:
#now format Average Price and Total Revenue in the summary table
summary_table_PA["Average Price"] = summary_table_PA["Average Price"].map("${:,.2f}".format)
summary_table_PA["Total Revenue"] = summary_table_PA["Total Revenue"].map("${:,.2f}".format)

summary_table_PA.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [57]:
gender_total = player_demo["Gender"].value_counts()
gender_percent = gender_total /  total_players
gender_df = pd.DataFrame({"Total Count": gender_total, "Percentage of Players": gender_percent})
gender_df


,Total Count,Percentage of Players
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [10]:
#determine gender classifications
purchase_data_df["Gender"].unique()

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

In [11]:
gender_groupby_df = purchase_data_df.groupby(["Gender"])
print(gender_groupby_df)
gender_groupby_df.count()

,Purchase ID,SN,Age,Item ID,Item Name,Price
Gender,,,,,,
Female,113,113,113,113,113,113
Male,652,652,652,652,652,652
Other / Non-Disclosed,15,15,15,15,15,15


In [19]:
gender_groupby_df["Purchase ID"].unique().count()


array([113, 652,  15], dtype=int64)

In [35]:
gender_groupby_df["SN"].count()

Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
Name: SN, dtype: int64

In [58]:
gender_groupby_df["SN"].unique()

Gender
Female                   [Lisassa64, Reunasu60, Reulae52, Assosia88, Ph...
Male                     [Lisim78, Lisovynya38, Ithergue48, Chamassasya...
Other / Non-Disclosed    [Chanosian48, Siarithria38, Haerithp41, Sundim...
Name: SN, dtype: object

In [60]:
print(len(gender_groupby_df["SN"].unique()))

3


In [40]:
male_players = purchase_data_df.loc[purchase_data_df["Gender"].value_counts()
total_male_players = len(male_players["SN"].unique())
print(total_male_players)

SyntaxError: invalid syntax (<ipython-input-40-0c17a9aff42f>, line 2)

In [42]:
gender_players_total = purchase_data_df["Gender"].value_counts()
gender_players_total

Series([], Name: Gender, dtype: int64)

In [14]:
#male_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Male", :]
#total_male_players = len(male_players["SN"].unique())
#total_male_players
#male_percent = (total_male_players) / (total_players) * 100
#male_percent

84.02777777777779

In [15]:
#female_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Female", :]
#total_female_players = len(female_players["SN"].unique())
#total_female_players
#female_percent = (total_female_players) / (total_players) * 100
#female_percent

14.0625

In [16]:
#other_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed", :]
#total_other_players = len(other_players["SN"].unique())
#total_other_players
#other_percent = (total_other_players) / (total_players) * 100
#other_percent

1.9097222222222223

In [33]:
#gender_summary = pd.DataFrame([{"Total Count": total_male_players, "Percentage of Players": male_percent},
                               {"Total Count": total_female_players, "Percentage of Players": female_percent},
                               {"Total Count": total_other_players, "Percentage of Players": other_percent}])
#gender_summary

,Total Count,Percentage of Players
0,484,84.027778
1,81,14.062500
2,11,1.909722


In [9]:
gender_analysis_df = purchase_data_df[["SN", "Gender", "Item Name", "Price"]]
gender_analysis_df.head()

,SN,Gender,Item Name,Price
0,Lisim78,Male,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,Male,Frenzied Scimitar,1.56
2,Ithergue48,Male,Final Critic,4.88
3,Chamassasya86,Male,Blindscythe,3.27
4,Iskosia90,Male,Fury,1.44


In [10]:
gender_analysis_df = gender_analysis_df.groupby(["Gender"])

ga_test = gender_analysis_df.count()
ga_test

,SN,Item Name,Price
Gender,,,
Female,113,113,113
Male,652,652,652
Other / Non-Disclosed,15,15,15


In [11]:
gender_analysis_df["SN"].unique()

Gender
Female                   [Lisassa64, Reunasu60, Reulae52, Assosia88, Ph...
Male                     [Lisim78, Lisovynya38, Ithergue48, Chamassasya...
Other / Non-Disclosed    [Chanosian48, Siarithria38, Haerithp41, Sundim...
Name: SN, dtype: object

In [12]:
#Gender Demographics (GD)
#find percent and count of: male players, female players and other/non-disclosed players

#male players
#create a dataframe removing females and other/non-disclosed
male_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Male", :]

#find unique number of male players
total_male_players = len(male_players_df["SN"].unique())

#percent of players which are male
male_percent = (total_male_players) / (total_players) * 100

male_players_df.head()



,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [13]:
#Gender Demographics (GD)
#female players

#create female dataframe
female_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Female", :]

#find total unique female players
total_female_players = len(female_players_df["SN"].unique())

female_percent = (total_female_players) / (total_players) * 100


female_players_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
15,15,Lisassa64,21,Female,98,"Deadline, Voice Of Subtlety",2.89
18,18,Reunasu60,22,Female,82,Nirvana,4.90
38,38,Reulae52,10,Female,116,Renewed Skeletal Katana,4.18
41,41,Assosia88,20,Female,7,"Thorn, Satchel of Dark Souls",1.33
55,55,Phaelap26,25,Female,84,Arcane Gem,3.79


In [14]:
#Gender Demographics (GD)
#other/non-disclosed players

#create other/non-disclosed dataframe
other_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed", :]

#find total unique other players
total_other_players = len(other_players_df["SN"].unique())


other_percent = (total_other_players) / (total_players) * 100

other_players_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58
22,22,Siarithria38,38,Other / Non-Disclosed,24,Warped Fetish,3.81
82,82,Haerithp41,16,Other / Non-Disclosed,160,Azurewrath,4.40
111,111,Sundim98,21,Other / Non-Disclosed,41,Orbit,4.75
228,228,Jiskirran77,20,Other / Non-Disclosed,80,Dreamsong,3.39


In [ ]:
m_f = pd.merge(male_players_df, female_players_df, on="Purchase ID")
gender_merge

In [ ]:
m_f_o = pd.merge(m_f, other_players_df, on="Purchase ID")
m_f_o